## Import

In [89]:
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline 

In [133]:
data = pd.read_csv("../Data/preprocessingData/K_20200414_preprocessing15.csv",encoding='CP949')
apartName = pd.read_csv("../Data/preprocessingData/K_20200414_PreprocessingPeriodApartName.csv",encoding='CP949')

In [82]:
data.tail()
apartName.tail()

,Apart
3563,중화동건영아파트
3564,청원다미소
3565,태능에셈빌
3566,한영(101)
3567,한영(104)


In [134]:
data=data.fillna(method='pad')

In [135]:
data=data.fillna(method='bfill')

In [136]:
print(data.columns.isin(apartName['Apart']).sum())
len(apartName['Apart'].unique())

3568


3568

In [137]:
data = data.set_index('date')

In [138]:
data = data[apartName['Apart'].unique()]

In [139]:
data.head()

,개포2차현대아파트(220),개포6차우성아파트1동~8동,개포우성3차,개포자이(12-2),개포주공 1단지,개포주공 3단지,개포주공 4단지,개포주공 5단지,개포주공 6단지,개포주공 7단지,...,화성두산아파트,동구햇살아파트101동,미영리치타운,미영리치타운103,범양프레체,중화동건영아파트,청원다미소,태능에셈빌,한영(101),한영(104)
date,,,,,,,,,,,,,,,,,,,,,
2,820.0,1052.0,807.0,830.0,1638,1782,1577,1206.0,1155,1178.0,...,445.0,436.0,418.0,412.0,652.0,323.0,419.0,358.0,331.0,330.0
3,820.0,1052.0,870.0,871.0,1653,1792,1611,1184.0,1193,1179.0,...,445.0,436.0,418.0,412.0,652.0,323.0,419.0,358.0,331.0,330.0
4,820.0,1025.0,856.0,860.0,1649,1764,1602,1184.0,1178,1194.0,...,472.0,436.0,425.0,412.0,652.0,323.0,419.0,390.0,331.0,330.0
5,820.0,1070.0,843.0,860.0,1666,1822,1662,1237.0,1160,1214.0,...,472.0,436.0,425.0,412.0,652.0,323.0,419.0,365.0,331.0,330.0
6,820.0,1013.0,843.0,875.0,1734,1872,1718,1202.0,1182,1192.0,...,478.0,436.0,425.0,412.0,652.0,342.0,419.0,377.0,331.0,330.0


In [141]:
log_data = np.log(data/data.shift(1))
log_data.head()

,개포2차현대아파트(220),개포6차우성아파트1동~8동,개포우성3차,개포자이(12-2),개포주공 1단지,개포주공 3단지,개포주공 4단지,개포주공 5단지,개포주공 6단지,개포주공 7단지,...,화성두산아파트,동구햇살아파트101동,미영리치타운,미영리치타운103,범양프레체,중화동건영아파트,청원다미소,태능에셈빌,한영(101),한영(104)
date,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.000000,0.075170,0.048216,0.009116,0.005596,0.021331,-0.018411,0.032371,0.000849,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,-0.026001,-0.016223,-0.012710,-0.002423,-0.015748,-0.005602,0.000000,-0.012653,0.012642,...,0.058905,0.0,0.016608,0.0,0.0,0.000000,0.0,0.085614,0.0,0.0
5,0.0,0.042966,-0.015303,0.000000,0.010257,0.032351,0.036769,0.043791,-0.015398,0.016612,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,-0.066249,0.0,0.0
6,0.0,-0.054742,0.000000,0.017291,0.040005,0.027073,0.033139,-0.028702,0.018788,-0.018288,...,0.012632,0.0,0.000000,0.0,0.0,0.057158,0.0,0.032348,0.0,0.0


In [144]:
log_data = log_data.dropna() 
log_data.head()

,개포2차현대아파트(220),개포6차우성아파트1동~8동,개포우성3차,개포자이(12-2),개포주공 1단지,개포주공 3단지,개포주공 4단지,개포주공 5단지,개포주공 6단지,개포주공 7단지,...,화성두산아파트,동구햇살아파트101동,미영리치타운,미영리치타운103,범양프레체,중화동건영아파트,청원다미소,태능에셈빌,한영(101),한영(104)
date,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.000000,0.075170,0.048216,0.009116,0.005596,0.021331,-0.018411,0.032371,0.000849,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,-0.026001,-0.016223,-0.012710,-0.002423,-0.015748,-0.005602,0.000000,-0.012653,0.012642,...,0.058905,0.000000,0.016608,0.0,0.0,0.000000,0.0,0.085614,0.0,0.0
5,0.0,0.042966,-0.015303,0.000000,0.010257,0.032351,0.036769,0.043791,-0.015398,0.016612,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,-0.066249,0.0,0.0
6,0.0,-0.054742,0.000000,0.017291,0.040005,0.027073,0.033139,-0.028702,0.018788,-0.018288,...,0.012632,0.000000,0.000000,0.0,0.0,0.057158,0.0,0.032348,0.0,0.0
7,0.0,0.006886,0.058739,0.067392,0.010898,0.068138,0.041055,0.055817,-0.094887,0.064958,...,-0.034046,0.031605,-0.043276,0.0,0.0,-0.023670,0.0,0.000000,0.0,0.0


In [146]:
mu = log_data.mean() 
mu

개포2차현대아파트(220)    0.007674
개포6차우성아파트1동~8동    0.001611
개포우성3차            0.011124
개포자이(12-2)        0.004664
개포주공 1단지          0.014083
                    ...   
중화동건영아파트          0.013647
청원다미소            -0.003136
태능에셈빌             0.010725
한영(101)          -0.002638
한영(104)           0.000000
Length: 3568, dtype: float64

In [213]:
mu_year = log_data.mean()*13
expect=pd.DataFrame(mu_year)
expect=expect.rename({0:'expect'},axis='columns')

In [216]:
risk=log_data.std() * np.sqrt(13)
risk = pd.DataFrame(risk)
risk=risk.rename({0:'risk'},axis='columns')

In [223]:
er=pd.concat([expect,risk],axis=1)
er

,expect,risk
개포2차현대아파트(220),0.099758,0.126664
개포6차우성아파트1동~8동,0.020946,0.093045
개포우성3차,0.144618,0.116537
개포자이(12-2),0.060628,0.167435
개포주공 1단지,0.183085,0.105601
...,...,...
중화동건영아파트,0.177408,0.111506
청원다미소,-0.040773,0.064594
태능에셈빌,0.139427,0.135037
한영(101),-0.034290,0.035584


In [220]:
apartName['pps']=pd.to_numeric(apartName['pps'])
apart = apartName.groupby(apartName['Apart']).mean()

In [228]:
per=pd.concat([apart,er],axis=1)
per

,pps,expect,risk
(1101-1),792.250000,0.132342,0.264974
(145-0),734.000000,0.051944,0.350263
(191-8)중앙궁전빌라트,366.000000,-0.127791,0.132615
(23-42),363.000000,0.000000,0.000000
(323-4),468.000000,0.116091,0.081962
...,...,...,...
힐타워,541.000000,0.116910,0.266030
힐탑,366.000000,0.057564,0.177707
힐탑아파트3차(894-3),293.000000,-0.101946,0.172712
힐탑트레져1동,1252.666667,-0.112313,0.165880


In [229]:
per.loc['힐탑']

pps       366.000000
expect      0.057564
risk        0.177707
Name: 힐탑, dtype: float64